# Lab 7

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/fayam1/Geospatial-Data-Management-course/blob/main/lab_07.ipynb)

## Submission requirements

1. Upload a screenshot of your map for each question.
2. Provide a link to your notebook on Colab. See instructions [here](https://geog-414.gishub.org/book/labs/instructions.html).

## Datasets

The datasets being used in the lab are listed below:

- [TIGER: US Census Counties](https://developers.google.com/earth-engine/datasets/catalog/TIGER_2018_Counties)
- [Landsat-9](https://developers.google.com/earth-engine/datasets/catalog/LANDSAT_LC09_C02_T1_L2)
- [Sentinel-2](https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S2_SR)
- [NAIP](https://developers.google.com/earth-engine/datasets/catalog/USDA_NAIP_DOQQ)

## Question 1

Create a fishnet with a 4-degree interval based on the extent of `[-112.5439, 34.0891, -85.0342, 49.6858]`. Use the fishnet to download the Landsat 7 image tiles by the fishnet using the `geemap.download_ee_image_tiles()` function. Relevant Earth Engine assets:

-   `ee.Image('LANDSAT/LE7_TOA_5YEAR/1999_2003')`



In [1]:
import geemap
import ee

# Initialize the Earth Engine API
ee.Initialize()

# Define the Landsat 7 image
image = ee.Image('LANDSAT/LE7_TOA_5YEAR/1999_2003')
vis_params = {'bands':['B4','B3','B2'],'min': 0, 'max': 255, 'gamma': 1.4}
# Define the extent as an Earth Engine geometry
extent = ee.Geometry.BBox(-112.5439, 34.0891, -85.0342, 49.6858)

# Create the fishnet
fishnet = geemap.fishnet(extent, rows=4, cols=4, delta=0.5)


# Optionally visualize the fishnet on the map (if you are using geemap.Map)
map = geemap.Map(center=[35,-100], zoom=4)
style = {'color': 'ffff00ff', 'fillColor': '00000000'}
map.addLayer(image, vis_params, 'Landsat')
map.addLayer(fishnet.style(**style), {}, 'Fishnet')
text = "Made by Faissal Bozi"
map.add_text(text, fontsize=20, position='bottomright')
map


Map(center=[35, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(ch…

In [ ]:
# Define the output directory
# I set scale=80 to save time
out_dir = 'Downloads'

# Download the Landsat 7 image tiles using the fishnet
geemap.download_ee_image_tiles(
    image, fishnet, out_dir, prefix="landsat_", crs="EPSG:3857", scale=80)

## Question 2

Create annual cloud-free Landsat imagery for the years 2017-2023 for a US county of your choice. Download the images to your computer. 



In [73]:
import geemap
import ee

# Initialize the Earth Engine API
ee.Initialize()

# Load the US Census Counties dataset
counties = ee.FeatureCollection("TIGER/2018/Counties")

# Filter for Knox County, Indiana
knox_county = counties.filter(ee.Filter.And(
    ee.Filter.eq('NAME', 'Knox'),
    ee.Filter.eq('STATEFP', '18')
))

geo = knox_county.geometry()

style = {
    'color': 'blue',  # Outline color
    'fillColor': '00000000',  # Transparent fill color
    'width': 2  # Outline width
}

# Applies scaling factors
def apply_scale_factors(image):
    optical_bands = image.select('SR_B.').multiply(0.0000275).add(-0.2)
    thermal_bands = image.select('ST_B.*').multiply(0.00341802).add(149.0)
    return image.addBands(optical_bands, None, True).addBands(thermal_bands, None, True)

# Create a Landsat 8 ImageCollection with cloud cover less than 5%
collection = ee.ImageCollection("LANDSAT/LC08/C02/T1_L2")\
             .filterBounds(knox_county)\
             .filterMetadata("CLOUD_COVER", "less_than", 5)\
             .map(apply_scale_factors)

# Visualization parameters
vis_params = {
    'bands': ['SR_B5', 'SR_B4', 'SR_B3'],  # False color  combination
    'min': 0.0,
    'max': 0.3,
    'gamma': 1.8,
}

# Create the time series
ts= geemap.create_timeseries(
    collection,
    start_date='2017-01-01',
    end_date='2023-12-31',
    region=geo,
    bands=['SR_B5', 'SR_B4', 'SR_B3'],
    frequency='year',
    reducer='median'
)

# Function to convert ImageCollection to a list of images
def collection_to_list(image_collection):
    size = image_collection.size().getInfo()
    return image_collection.toList(size)

# Convert the ImageCollection to a list
ts_list = collection_to_list(ts)

# Create a map
map1 = geemap.Map()

# Add each image in the time series to the map
for i in range(ts.size().getInfo()):
    image = ee.Image(ts_list.get(i))
    year = 2017 + i
    map1.addLayer(image, vis_params, f'Knox County {year}')

# Display the map
map1.centerObject(geo, 10)
map1.addLayer(knox_county .style(**style),{},'Knox County')
map1.addLayerControl()  # This will add a layer control panel to the map
#Add text
text = "Made by Faissal Bozi"
map1.add_text(text, fontsize=20, position='bottomright')
map1


Map(center=[38.688923758741744, -87.41827504818303], controls=(WidgetControl(options=['position', 'transparent…

In [91]:
# Export each image in the time series
for i in range(ts.size().getInfo()):
    image = ee.Image(ts_list.get(i))
    year = 2017 + i
    out_file = f'Landsat_Knox_County_{year}.tif'
    geemap.ee_export_image(image.clip(geo), filename=out_file, scale=80, region=geo)

print("Images saved successfully.")

Generating URL ...
Please wait ...
Data downloaded to C:\Users\musao\Desktop\Data management using google earth engine\Geospatial-Data-Management-course\Landsat_Knox_County_2017.tif
Generating URL ...
Please wait ...
Data downloaded to C:\Users\musao\Desktop\Data management using google earth engine\Geospatial-Data-Management-course\Landsat_Knox_County_2018.tif
Generating URL ...
Please wait ...
Data downloaded to C:\Users\musao\Desktop\Data management using google earth engine\Geospatial-Data-Management-course\Landsat_Knox_County_2019.tif
Generating URL ...
Please wait ...
Data downloaded to C:\Users\musao\Desktop\Data management using google earth engine\Geospatial-Data-Management-course\Landsat_Knox_County_2020.tif
Generating URL ...
Please wait ...
Data downloaded to C:\Users\musao\Desktop\Data management using google earth engine\Geospatial-Data-Management-course\Landsat_Knox_County_2021.tif
Generating URL ...
Please wait ...
Data downloaded to C:\Users\musao\Desktop\Data manageme

## Question 3

Create annual cloud-free Sentinel-2 imagery for the years 2017-2023 for a US county of your choice. Download the images to your computer. You can download a coarse resolution image to speed up the download process. Narrow down the date range (e.g., summer months) to reduce the number of images, which can avoid memory errors.


In [92]:
import geemap
import ee

# Initialize the Earth Engine API
ee.Initialize()

# Load the US Census Counties dataset
counties = ee.FeatureCollection("TIGER/2018/Counties")

# Filter for Knox County, Indiana
knox_county = counties.filter(ee.Filter.And(
    ee.Filter.eq('NAME', 'Knox'),
    ee.Filter.eq('STATEFP', '18')
))

geo = knox_county.geometry()

style = {
    'color': 'blue',  # Outline color
    'fillColor': '00000000',  # Transparent fill color
    'width': 2  # Outline width
}

# Applies scaling factors
def apply_scale_factors(image):
    optical_bands = image.select(['B2', 'B3', 'B4', 'B8']).multiply(0.0001)
    return image.addBands(optical_bands, None, True)

# Create a Sentinel-2 ImageCollection with cloud cover less than 5%
collection1 = ee.ImageCollection("COPERNICUS/S2_SR")\
             .filterBounds(knox_county)\
             .filterMetadata("CLOUDY_PIXEL_PERCENTAGE", "less_than", 5)\
             .map(apply_scale_factors)

# Visualization parameters for true color
vis = {
    'bands': ['B4', 'B3', 'B2'],  # True color (Red, Green, Blue) combination
    'min': 0.0,
    'max': 0.3,
    'gamma': 1.2,
}

# Create the time series for the months from May to August 2023 to save time
ts1 = geemap.create_timeseries(
    collection1,
    start_date='2023-05-01',
    end_date='2023-08-31',
    region=geo,
    bands=['B4', 'B3', 'B2'],
    frequency='month',
    reducer='median'
)

# Function to convert ImageCollection to a list of images
def collection1_to_list(image_collection):
    size = image_collection.size().getInfo()
    return image_collection.toList(size)

# Convert the ImageCollection to a list
ts1_list = collection1_to_list(ts1)

# Create a map
map2 = geemap.Map()

# Define the month names
month_names = ['May', 'June', 'July', 'August']

# Add each image in the time series to the map
for i in range(min(ts1.size().getInfo(), len(month_names))):
    image1 = ee.Image(ts1_list.get(i))
    month_name = month_names[i]
    map2.addLayer(image1, vis, f'Knox County {month_name} 2023')

# Display the map
map2.centerObject(geo, 10)
map2.addLayer(knox_county.style(**style), {}, 'Knox County')
map2.addLayerControl()  # This will add a layer control panel to the map

# Add text
text = "Made by Faissal Bozi"
map2.add_text(text, fontsize=20, position='bottomright')

map2



Map(center=[38.688923758741744, -87.41827504818303], controls=(WidgetControl(options=['position', 'transparent…

In [89]:
# Export each image in the time series
for i in range(min(ts1.size().getInfo(), len(month_names))):
    image1 = ee.Image(ts1_list.get(i))
    month_name = month_names[i]
    out_file = f'Sentinel2_Knox_County_{month_name}_2023.tif'
    geemap.ee_export_image(image1, filename=out_file, scale=80, region=geo)

print("Images saved.")



Generating URL ...
Please wait ...
Data downloaded to C:\Users\musao\Desktop\Data management using google earth engine\Geospatial-Data-Management-course\Sentinel2_Knox_County_May_2023.tif
Generating URL ...
Please wait ...
Data downloaded to C:\Users\musao\Desktop\Data management using google earth engine\Geospatial-Data-Management-course\Sentinel2_Knox_County_June_2023.tif
Generating URL ...
Please wait ...
Data downloaded to C:\Users\musao\Desktop\Data management using google earth engine\Geospatial-Data-Management-course\Sentinel2_Knox_County_July_2023.tif
Generating URL ...
Please wait ...
Data downloaded to C:\Users\musao\Desktop\Data management using google earth engine\Geospatial-Data-Management-course\Sentinel2_Knox_County_August_2023.tif
Images saved.


## Question 4

Create annual cloud-free NAIP imagery for the years 2010-2023 for a US county of your choice. Download the images to your computer. You can download a coarse resolution image to speed up the download process. 



In [2]:
import geemap
import ee

# Initialize the Earth Engine API
ee.Initialize()

# Load the US Census Counties dataset
counties = ee.FeatureCollection("TIGER/2018/Counties")

# Filter for Knox County, Indiana
knox_county = counties.filter(ee.Filter.And(
    ee.Filter.eq('NAME', 'Knox'),
    ee.Filter.eq('STATEFP', '18')
))

geo = knox_county.geometry()

style = {
    'color': 'blue',  # Outline color
    'fillColor': '00000000',  # Transparent fill color
    'width': 2  # Outline width
}

# Create a NAIP ImageCollection
collection2 = ee.ImageCollection("USDA/NAIP/DOQQ")\
             .filterBounds(knox_county)\
             .filterDate('2010-01-01', '2023-12-31')

# Visualization parameters for true color
vis = {
    'bands': ['R', 'G', 'B'],  # True color (Red, Green, Blue) combination
    'min': 0.0,
    'max': 255,
    'gamma': 1.2,
}

# Create the time series for the years from 2010 to 2023
ts2 = geemap.create_timeseries(
    collection2,
    start_date='2010-01-01',
    end_date='2023-12-31',
    region=geo,
    bands=['R', 'G', 'B'],
    frequency='year',
    reducer='median'
)

# Function to convert ImageCollection to a list of images
def collection1_to_list(image_collection):
    size = image_collection.size().getInfo()
    return image_collection.toList(size)

# Convert the ImageCollection to a list
ts2_list = collection1_to_list(ts2)

# Create a map
map3 = geemap.Map()

# Add each image in the time series to the map
for i in range(ts2.size().getInfo()):
    image2 = ee.Image(ts2_list.get(i))
    year = 2010 + i
    map3.addLayer(image2, vis, f'Knox County {year}')

# Display the map
map3.centerObject(geo, 10)
map3.addLayer(knox_county.style(**style), {}, 'Knox County')
map3.addLayerControl()  # This will add a layer control panel to the map

# Add text
text = "Made by Faissal Bozi"
map3.add_text(text, fontsize=20, position='bottomright')

map3




Map(center=[38.688923758741744, -87.41827504818303], controls=(WidgetControl(options=['position', 'transparent…

In [ ]:
# Export each image in the time series
for i in range(ts2.size().getInfo()):
    image2 = ee.Image(ts2_list.get(i))
    year = 2010 + i
    out_file = f'NAIP_Knox_County_{year}.tif'
    geemap.ee_export_image(image1, filename=out_file, scale=100, region=geo)

print("Images saved.")

## Question 5

Download a US county of your choice and save it as a shapefile or GeoJSON file. 



In [14]:
# Add your code here.
import geemap
import ee

# Initialize the Earth Engine API
ee.Initialize()

# Load the US Census Counties dataset
counties = ee.FeatureCollection("TIGER/2018/Counties")

# Filter for Knox County, Indiana
knox_county = counties.filter(ee.Filter.And(
    ee.Filter.eq('NAME', 'Knox'),
    ee.Filter.eq('STATEFP', '18')
))

style = {
    'color': 'blue',  # Outline color
    'fillColor': '00000000',  # Transparent fill color
     'fillOpacity': 0 ,
    'width': 2  # Outline width
}

# Export Knox County geometry to a shapefile
geemap.ee_export_vector(knox_county, filename='Knox_County.shp')

# Create a map
map4 = geemap.Map()

# Add Knox County to the map
map4.add_shp('Knox_County.shp',style=style, layer_name='Knox County')

# Display the map
map4.centerObject(knox_county)
# Add text
text = "Made by Faissal Bozi"
map4.add_text(text, fontsize=20, position='bottomright')
map4



Generating URL ...
Please wait ...
An error occurred while downloading. 
 Retrying ...
Generating URL ...
Please wait ...
Data downloaded to C:\Users\musao\Desktop\Data management using google earth engine\Geospatial-Data-Management-course\Knox_County.shp


Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…